In [1]:
import os
import librosa
import glob
import json
import numpy as np
from preprocess import get_allpaths, preprocess
from utils      import load_config, load_logger, load_parallel_pool, generateOutputCQTList

In [2]:
config = load_config()
logger = load_logger()
c      = load_parallel_pool()

/home/mirlab/miniconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@mirlab')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [3]:
def create_db_refs(ref_file, artist):
    import librosa 
    import os
    import numpy as np
    
    audio_file = os.path.join(config['AUDIO_DIR'], ref_file + '.wav')
    y, sr = librosa.load(audio_file)
    Q = librosa.cqt(y, sr=sr, fmin=130.81, n_bins=121, bins_per_octave=24, hop_length=96)
    
    # Preprocess CQT to logQ
    ds = 3
    absQ = np.absolute(Q)
    smoothQ = np.zeros((absQ.shape[0], absQ.shape[1]//ds))
    for row in range(absQ.shape[0]):
        smoothQ[row] = np.convolve(absQ[row], [1/ds]*ds, 'valid')[0:absQ.shape[1]-ds+1:ds]
    logQ = np.log(1+1000000*smoothQ)
    
    output_file = os.path.join(config["DATA_OUTPUT_DIR"], ref_file + '.npy')
    np.save(output_file, logQ)
    return None

In [8]:
for artist in config['artists']:
    ref_files = get_allpaths(artist, os.path.join(config['AUDIO_DIR'], 'Lists/'))
    c[:].push(dict(config=config, 
                   artist=artist,
                   create_db_refs=create_db_refs))
    output = c[:].map(lambda x: create_db_refs(x, artist), ref_files)
    output.wait_interactive()
    logger.debug("Finish loading CQT for {:} ({:}s taken)".format(artist, output.elapsed))
    
    f = open(generateOutputCQTList(config["DATA_OUTPUT_DIR"], artist), 'w')
    for file in ref_files:
        f.write(file + '.npy\n')
    f.close()

  12/12 tasks finished after   82 s
done
2018-03-10 18:43:30,654 - root - DEBUG - Finish loading CQT for foofighters (82.750434s taken)
